In [1]:
import pandas as pd 
import numpy as np 
import pickle
import re 

In [5]:
word_token = {}
with open('word_token.pkl', 'rb') as file:
    # Call load method to deserialze
    word_token = pickle.load(file)

token_word = {}
with open('token_word.pkl', 'rb') as file:
    token_word = pickle.load(file)

word_token["up"], token_word[0]

(4, 'Jan 03, 1984')

In [6]:
df = pd.read_csv("Ftse100janapr21.csv")
df = df.reindex(index=df.index[::-1])
columns = df.columns
df.head()

,Date,Open,High,Low,Close_,AdjClose__,Volume
71,"Jan 04, 2021","6,460.50","6,662.70","6,460.50","6,571.90","6,571.90","965,097,800"
70,"Jan 05, 2021","6,571.90","6,622.80","6,544.30","6,612.30","6,612.30","718,950,800"
69,"Jan 06, 2021","6,612.30","6,859.10","6,611.90","6,841.90","6,841.90","1,249,709,500"
68,"Jan 07, 2021","6,841.90","6,903.60","6,795.10","6,857.00","6,857.00","939,652,600"
67,"Jan 08, 2021","6,857.00","6,886.30","6,836.10","6,873.30","6,873.30","850,778,500"


In [7]:
# convert all the values from strings to floats. 
def string_to_float(x):
    if x is not np.nan:
        x = re.sub(",", "",x)
        x = float(x)
    else:
        x = 0 
    return x

for col in columns:
    if col != "Date":
        df[col] = df[col].apply(lambda x: string_to_float(str(x)))



In [8]:
def add_moving_average(term, exp=False):
    for col in columns:
        if col != "Date" and col != "target":
            if exp:
                col_name = col + str(term) + "day" + "_exp"
                df[col_name] = df[col].rolling(window=term, win_type='triang').mean()
            else: 
                col_name = col + str(term) + "day"
                df[col_name] = df[col].rolling(window=term).mean()

In [9]:
flat_averages = [10,32,45,78,101]
exp_averages  = [3,12,48,112]

In [10]:
for x in flat_averages: 
    add_moving_average(x)
for x in exp_averages:
    add_moving_average(x, exp=True)

In [11]:
df["target"] = df["Close_"].shift(1)

In [12]:
def word_target(x):
    y = "up"
    if x['Close_'] > x["target"]:
        y = "down"  
    return y 
df["wordT"] = df.apply(lambda x: word_target(x), axis=1)

In [27]:
df.head()

,Date,Open,High,Low,Close_,AdjClose__,Volume,Open10day,High10day,Low10day,...,AdjClose__48day_exp,Volume48day_exp,Open112day_exp,High112day_exp,Low112day_exp,Close_112day_exp,AdjClose__112day_exp,Volume112day_exp,target,wordT
71,"Jan 04, 2021",6460.5,6662.7,6460.5,6571.9,6571.9,9.650978e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,up
70,"Jan 05, 2021",6571.9,6622.8,6544.3,6612.3,6612.3,7.189508e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6571.9,down
69,"Jan 06, 2021",6612.3,6859.1,6611.9,6841.9,6841.9,1.249710e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6612.3,down
68,"Jan 07, 2021",6841.9,6903.6,6795.1,6857.0,6857.0,9.396526e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6841.9,down
67,"Jan 08, 2021",6857.0,6886.3,6836.1,6873.3,6873.3,8.507785e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6857.0,down


In [14]:
pred_strings = {}

In [15]:
def add_to_string(y,x):
    global pred_strings
    if y in pred_strings.keys():
        if x == "\n":
            pred_strings[y] += x 
        else:
            x = str(x) + " ; "
            pred_strings[y] += x
    else:
        x = str(x) + " ; "
        pred_strings[y] = x

In [17]:
for i, row in df.iterrows():
    for col in df.columns:
        if col != "target" and col != "wordT":
            add_to_string(i,row[col])
    add_to_string(i,"\n")


In [18]:
pred_strings[0]

'Apr 15, 2021 ; 6939.6 ; 6994.9 ; 6939.3 ; 6983.5 ; 6983.5 ; 651402900.0 ; 6850.910000000001 ; 6892.449999999995 ; 6831.279999999997 ; 6872.05 ; 6872.05 ; 679724300.0 ; 6747.481250000002 ; 6789.518749999998 ; 6713.328124999998 ; 6763.109375000002 ; 6763.109375000002 ; 747430240.625 ; 6713.33555555555 ; 6755.868888888888 ; 6673.604444444443 ; 6723.377777777771 ; 6723.377777777771 ; 785870631.1111112 ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; nan ; 6902.424999999999 ; 6943.65 ; 6890.099999999999 ; 6938.3 ; 6938.3 ; 604385650.0 ; 6835.909722222224 ; 6881.606944444445 ; 6818.045833333335 ; 6861.350000000002 ; 6861.350000000002 ; 692086811.1111112 ; 6696.752083333335 ; 6738.457031250002 ; 6653.1165798611155 ; 6703.794097222225 ; 6703.794097222225 ; 805093230.2951391 ; nan ; nan ; nan ; nan ; nan ; nan ; \n'

In [19]:
pred_sentance_tokens = {}
token = max(token_word.keys())
token

306997

In [22]:
for k in pred_strings.keys():
    s = pred_strings[k].split(";")
    st = []
    for x in s:
        if x in word_token.keys():
            st.append(word_token[x])
        else:
            st.append(token)
            token += 1 
    pred_sentance_tokens[k] = st

In [29]:
pred_sentance_tokens[70], token

([307059,
  307060,
  307061,
  307062,
  307063,
  307064,
  307065,
  307066,
  307067,
  307068,
  307069,
  307070,
  307071,
  307072,
  307073,
  307074,
  307075,
  307076,
  307077,
  307078,
  307079,
  307080,
  307081,
  307082,
  307083,
  307084,
  307085,
  307086,
  307087,
  307088,
  307089,
  307090,
  307091,
  307092,
  307093,
  307094,
  307095,
  307096,
  307097,
  307098,
  307099,
  307100,
  307101,
  307102,
  307103,
  307104,
  307105,
  307106,
  307107,
  307108,
  307109,
  307110,
  307111,
  307112,
  307113,
  307114,
  307115,
  307116,
  307117,
  307118,
  307119,
  307120],
 311461)